In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re
import os
import pickle
from tqdm import tqdm
from constants import base_path, app_n_questions, model_list, pathology_scope, positive_threshold
import matplotlib.pyplot as plt
from collections import Counter
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
tqdm.pandas()

c:\Users\myras\Documents\MYMY\ACADS\AY 2023-2024 2ND SEM\STAT299 - Capstone\Repo\VITAS - 5 pathology\vitas-env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with open(f"{base_path}\\input\\release_conditions.json") as f:
  disease_dict = json.load(f)
if pathology_scope:
   disease_list =  pathology_scope
else:
  disease_list = list(disease_dict.keys())

In [3]:
with open(f"{base_path}\\input\\release_evidences.json") as f:
  evidences = json.load(f)
evidences_list = []
evidences_dict = {}
evidences_en_to_code = {}
for e in evidences.keys():
  # only binary symptoms and antecedents
  if (not evidences[e]["possible-values"]):
    evidences_list.append(e)
    evidences_dict[e] = evidences[e]["question_en"]
    evidences_en_to_code[evidences[e]["question_en"]] = e
evidences_code_to_en = evidences_dict
evidences_list_en = list(evidences_en_to_code.keys())
evidences_dict["AGE"] = "AGE"
evidences_dict["SEX"] = "SEX"
feature_columns = ["AGE", "SEX"] + evidences_list

In [4]:
def get_next_question(evidences, questionnaire, feature_embeddings_df):
    centroid = np.array([feature_embeddings_df.loc[e].values for e in evidences]).mean(axis=0)
    _, indices = questionnaire.kneighbors([centroid])
    ask_list = [evidences_list_en[i] for i in indices[0] if evidences_list_en[i] not in evidences]
    try:
        return ask_list
    except:
        return []

In [5]:
def get_evidences(answers, user_evidences, questionnaire, feature_embeddings_df):
    ask = True
    question_counter = 1 # counts initial evidence sa q0
    evidences_en = [evidences_code_to_en[e] for e in user_evidences]
    answers_en = [evidences_code_to_en[e] for e in answers if e in evidences_list]
    asked = evidences_en.copy()
    while question_counter < app_n_questions:
      ask =  True
      next_question_idx = 0
      next_question = get_next_question(evidences_en, questionnaire, feature_embeddings_df)
      while ask and (question_counter < app_n_questions):
        if next_question[next_question_idx] not in asked:
          answer = 1 if next_question[next_question_idx] in answers_en else 0
          asked.append(next_question[next_question_idx])
          question_counter+=1
          if answer==1:
            evidences_en.append(next_question[next_question_idx])
            ask = False
          else:
            next_question_idx += 1
          if next_question_idx > app_n_questions:
            break
        else:
            next_question_idx += 1
    return [evidences_en_to_code[e] for e in evidences_en]

In [6]:
def data_proc(df, questionnaire, feature_embeddings_df):
    df["binary_evidences_all"] = df["EVIDENCES"].apply(lambda x: [d for d in ast.literal_eval(x) if d in evidences_list])
    df["binary_evidences_all_count"] = df["binary_evidences_all"].apply(lambda x: len(x))
    df["binary_evidences"] = df[["EVIDENCES", "INITIAL_EVIDENCE"]].progress_apply(lambda x: get_evidences(ast.literal_eval(x[0]), [x[1]], questionnaire, feature_embeddings_df), axis=1)
    df["binary_evidences_count"] = df["binary_evidences"].apply(lambda x: len(x))
    df["hit_rate"] = df["binary_evidences_count"]/df["binary_evidences_all_count"]
    hit_rate = df["hit_rate"]
    for e in evidences_list:
        df[e] = df["binary_evidences"].apply(lambda x: 1 if e in x else 0)
    df["SEX"] = df["SEX"].map({'F': 0, 'M': 1})
    ftr_df = df[feature_columns + ["PATHOLOGY"]]
    questionnaire_df = df[["binary_evidences_all", "binary_evidences", "INITIAL_EVIDENCE"]]
    return ftr_df, hit_rate, questionnaire_df

In [8]:
def pred(x):
    pred_list = []
    for i in range(len(disease_list)):
        if x[i] >= positive_threshold:
            pred_list.append({
                "disease": disease_list[i],
                "probability": x[i]})
    if pred_list:
        pred_df = pd.DataFrame(pred_list).set_index('disease')
        # return only top 1 - allows ties
        pred_df['rank'] = pred_df['probability'].rank(method='min', ascending=False)
        pred_df = pred_df.sort_values(by="rank")
        pred_df = pred_df[pred_df["rank"]<=1][["probability"]]
        if pred_df.shape[0] > 1: # in case of tied rankings
            pred_df = pred_df.sample(random_state=1)
        return list(pred_df.index)
    else:
        return []

In [9]:
def get_missed_evidence(actual, asked):
    return list(set(actual)^set(asked))

In [10]:
def validate(x):
    if x[0]:
        return [x[0]]==x[1]
    else:
        return not x[1]

In [11]:
with open(f'{base_path}\\output\\questionnaire\\questionnaire.pkl', 'rb') as f:
    questionnaire = pickle.load(f)
feature_embeddings_df = pd.read_pickle(f'{base_path}\\output\\questionnaire\\questionnaire_embeddings.pkl')

In [12]:
diagnosis_df_valid = pd.read_csv(f"{base_path}\\input\\release_validate_patients")
diagnosis_df_valid["PATHOLOGY"] = [i if i in disease_list else "" for i in diagnosis_df_valid["PATHOLOGY"]]
diagnosis_df_valid

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,55,"[['Anemia', 0.25071110167158567], ['Atrial fib...",F,,"['E_7', 'E_24', 'E_26', 'E_53', 'E_54_@_V_180'...",E_154
1,10,"[['Guillain-Barré syndrome', 0.135558991316712...",F,,"['E_16', 'E_29', 'E_50', 'E_53', 'E_54_@_V_182...",E_171
2,68,"[['Influenza', 0.1900250899717378], ['Viral ph...",F,,"['E_50', 'E_53', 'E_54_@_V_183', 'E_54_@_V_198...",E_53
3,13,"[['Anemia', 0.18697604010451876], ['Atrial fib...",M,,"['E_7', 'E_24', 'E_26', 'E_53', 'E_54_@_V_180'...",E_53
4,48,"[['Boerhaave', 1.0]]",M,,"['E_53', 'E_54_@_V_71', 'E_54_@_V_112', 'E_54_...",E_53
...,...,...,...,...,...,...
132443,27,"[['Viral pharyngitis', 0.22702125813983617], [...",M,,"['E_41', 'E_48', 'E_53', 'E_54_@_V_161', 'E_55...",E_201
132444,57,"[['Acute pulmonary edema', 0.12078088376840804...",M,,"['E_5', 'E_53', 'E_54_@_V_154', 'E_54_@_V_183'...",E_151
132445,52,"[['GERD', 0.24494427036287517], ['Bronchitis',...",F,GERD,"['E_53', 'E_54_@_V_112', 'E_54_@_V_161', 'E_54...",E_173
132446,10,"[['Epiglottitis', 0.2969684152571116], ['HIV (...",M,,"['E_53', 'E_54_@_V_179', 'E_54_@_V_192', 'E_55...",E_91


In [13]:
# Get features from questionnaire
diagnosis_df_valid, hit_rate, questionnaire_df = data_proc(diagnosis_df_valid, questionnaire, feature_embeddings_df)
questionnaire_df["missed_evidence"] = questionnaire_df.progress_apply(lambda x: get_missed_evidence(x[0], x[1]), axis=1)
questionnaire_df_path = f"{base_path}\\output\\error_analysis_questionnaire"
if not os.path.exists(questionnaire_df_path):
    os.makedirs(questionnaire_df_path)
questionnaire_df.to_csv(f"{questionnaire_df_path}\\questionnaire_df.csv")
missed_evidences = []
for e in questionnaire_df["missed_evidence"]:
    if e:
        missed_evidences.extend(e)
missed_evidences_dict = dict(Counter(missed_evidences).most_common(10))
missed_evidences_dict = {evidences_code_to_en[k]:missed_evidences_dict[k] for k in missed_evidences_dict}
with open(f"{questionnaire_df_path}\\top_missed_evidences.json", "w") as outfile: 
    json.dump(missed_evidences_dict, outfile, indent=True)

100%|██████████| 132448/132448 [00:02<00:00, 55501.06it/s]


In [14]:
def save_preds(model_name):
    print(f"Evaluating {model_name}...")
    counter = 0
    for disease in disease_list:
        disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
        with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{model_name}\\{disease_filename}_model.pkl', 'rb') as f:
            clf_model = pickle.load(f)
        diagnosis_df_valid[disease] = np.round(clf_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1], 2)
        counter+=1
        print(f"done {counter}: {disease}")
    diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)
    diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)
    diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis", "is_matched"] + disease_list].to_csv(f"{questionnaire_df_path}\\{model_name}\\validation_df_all_patients_questionnaire.csv")

## Tree-based models

In [15]:
for model_name in model_list["tree-based"]:
    save_preds(model_name)

Evaluating decision_tree...
done 1: Tuberculosis
done 2: GERD
done 3: SLE
done 4: HIV (initial infection)
done 5: Pulmonary neoplasm


100%|██████████| 132448/132448 [00:01<00:00, 80253.03it/s]


Evaluating random_forest...
done 1: Tuberculosis
done 2: GERD
done 3: SLE
done 4: HIV (initial infection)
done 5: Pulmonary neoplasm


100%|██████████| 132448/132448 [00:02<00:00, 57399.70it/s]


Evaluating gradient_boost...
done 1: Tuberculosis
done 2: GERD
done 3: SLE
done 4: HIV (initial infection)
done 5: Pulmonary neoplasm


100%|██████████| 132448/132448 [00:01<00:00, 83794.18it/s]


## Logistic Regression

In [16]:
save_preds("logistic_regression")

Evaluating logistic_regression...
done 1: Tuberculosis
done 2: GERD
done 3: SLE
done 4: HIV (initial infection)
done 5: Pulmonary neoplasm


100%|██████████| 132448/132448 [00:01<00:00, 86575.25it/s]
